<a href="https://colab.research.google.com/github/Elimirage/deep_learning_24vvim1/blob/main/lab8/lab8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Любая LLM работает с токенами, поэтому нам нужна не только модель, но и токенизатор
from transformers import AutoModelForCausalLM, AutoTokenizer

In [ ]:
# Зададим название модели из репозитория huggingface
# список доступных моделей можно посмотреть по ссылке https://huggingface.co/models
# будем использовать модель Qwen или YandexGPT

model_name = "Qwen/Qwen2.5-7B-Instruct-1M"
#model_name = "yandex/YandexGPT-5-Lite-8B-instruct"

# В названии модели обычно указываются - версия (2,5) количество параметров (7 млрд.)
# этап обучения или предназначение (Base, Instruct, Code, Thinking и т.п.)
# размер контекстного окна (1 млн.)
# также могут указываться параметры квантования (FP8, GGUF, ) и другие характеристики.


# Загружаем предобученную можель
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype="auto",
    device_map="auto"
)

# Загружаем предобученный токенизатор
tokenizer = AutoTokenizer.from_pretrained(model_name)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/825 [00:00<?, ?B/s]

`torch_dtype` is deprecated! Use `dtype` instead!


model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

...

generation_config.json:   0%|          | 0.00/243 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

In [ ]:
file = 'ENG_article.txt'
with open(file, 'r', encoding='utf-8') as file:
    data = file.read().replace('\n', ' ')

print(f"Длина текста: {len(data)} символов")
print(f"Количество слов: {len(data.split())}")

Длина текста: 8291 символов
Количество слов: 1157


In [ ]:
messages = [
    {"role": "system", "content": "Проанализируй текст и найди точные ответы на вопросы. Отвечай кратко и только по фактам."},
    {"role": "user", "content":
     f"""
     Текст: {data[:15000]}

     Вопросы:
     1. В каком году была обозначена проблема взрывающихся градиентов?
     2. Кто в 1891 году разработал метод уничтожающей производной?
     3. Кто предложил цепное правило дифференцирования и в каком году?
     """}
]


text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True
)

print("Промпт:")
print(text[:500])

Промпт:
<|im_start|>system
Проанализируй текст и найди точные ответы на вопросы. Отвечай кратко и только по фактам.<|im_end|>
<|im_start|>user

     Текст: Wikipedia https://en.m.wikipedia.org/wiki/Neural_network_(machine_learning) Neural network (machine learning)     This article is about the computational models used for artificial intelligence. For other uses, see Neural network (disambiguation).  Typically, neurons are aggregated into layers. Different layers may perform different transformations o


In [ ]:
model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

generated_ids = model.generate(
    **model_inputs,
    max_new_tokens=300,
    temperature=0.7
)

generated_ids_ = [
    output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
]

response = tokenizer.batch_decode(generated_ids_, skip_special_tokens=True)[0]

In [ ]:
print(response)

1. В 1991 году была обозначена проблема взрывающихся градиентов.

2. Вопрос о методе уничтожающей производной не имеет отношения к тексту и год 1891 года не упоминается.

3. Готфрид Вильhelm Лейбниц предложил цепное правило дифференцирования в 1673 году.
